Consulta o banco de dados e salva em csv cada tabela

In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd
from datetime import timedelta
import os

save_path = r"pasta para salvar arquivos"
# Database connection
engine = create_engine(
    "endereço base", 
    echo=False
)

# Defina o limite de tempo
tempo_limite = pd.Timestamp.now().floor('s') - timedelta(minutes=30)

# Query to get table names
table_query = text(
    "SELECT tablename FROM pg_tables WHERE tablename LIKE '%_POS%' AND tablename NOT LIKE '%OLD%' AND schemaname = 'public'"
)

with engine.connect() as conn:
    # Execute the query to get table names
    table_names = conn.execute(table_query).fetchall()

    # Iterate through table names and export each to a CSV file
    for table_name in table_names:
        print(table_name)
        table_name = table_name[0]  # Get the table name string from the result
        query = text(f'SELECT "TIMESTAMP", "GPOA" FROM "{table_name}"  WHERE "TIMESTAMP" >= :tempo_limite')
        df = pd.read_sql(query, conn, params={'tempo_limite': tempo_limite})  # Read the table into a DataFrame
        csv_file_name = f"\\{table_name.replace('/', '_')}.csv"
        final_save_path = save_path + csv_file_name        

        df.to_csv(final_save_path, index=False)  # Save DataFrame to CSV

print("CSV files have been created.")

